## Mount the Drive, and Change to Google Drive Folder

In [1]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

# %cd /content/drive/MyDrive/MSc.-Dissertations/1/Files
# %ls

## Import Libraries

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import models, layers, utils, losses
from keras.wrappers import scikit_learn
from keras.models import *
from keras.layers import *
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.utils import np_utils

from sklearn.decomposition import PCA, SparsePCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

import random
import pandas as pd
import numpy as np
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt

## Count the Number of Files, and Take Random Samples from the Image Files

In [3]:
# !ls street_view
# count how many files and write the filenames into a file
# !ls street_view -1 | wc -l 
# !ls street_view/*.jpg > flist.txt
flist_old = list(pd.read_csv('flist.txt', header = None)[0])
flist = []
change_names = list(map(lambda x: flist.append(f'street_view/{x}'), flist_old))

# Set seed so sample is reproducible 
# random.seed(99)  # set this to an integer value!!!
# nsamp = 100
# flist_sub = random.sample(flist, nsamp)
# flist = flist_sub

# print(flist)

## Overview of the `properties` Dataset

Read the `properties` dataset first, and make sure that `property type` is a categorical variable.

In [4]:
properties = pd.read_csv('properties.csv')
properties_juny12 = pd.read_csv('properties_juny12.csv')
properties_full = pd.concat([properties, properties_juny12])
properties = properties_full
properties.propertyType = properties.propertyType.astype('category')
properties.head()

,Unnamed: 0,address,propertyType,bedrooms,detailUrl,location_lat,location_lng,property_id
0,0,"12, Gorsey Brigg, Dronfield Woodhouse, Dronfie...",Terraced,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29986,-1.49446,60d9dd15-c5a0-4d9c-a341-a1d47add49d5
1,0,"5, Highgate Lane, Dronfield, Derbyshire S18 1UB",Detached,4.0,https://www.rightmove.co.uk/house-prices/detai...,53.29135,-1.45975,4a586e80-181a-4b82-b5c3-2d789436bb14
2,0,"125, Gosforth Lane, Dronfield, Derbyshire S18 1RB",Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29763,-1.47573,93680b6c-237e-44d3-8f40-959a14b80cad
3,0,"80, Shakespeare Crescent, Dronfield, Derbyshir...",Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29259,-1.45644,5d49758b-f148-4d06-bbae-3eb23f5c68fb
4,0,"21, Gainsborough Road, Dronfield, Derbyshire S...",Detached,NaN,https://www.rightmove.co.uk/house-prices/detai...,53.29740,-1.48503,4645f5eb-de7c-474f-8d7e-b59fa8c55f19


Basic information of the dataset is shown as follows.

In [5]:
properties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37402 entries, 0 to 19851
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Unnamed: 0    37402 non-null  int64   
 1   address       37402 non-null  object  
 2   propertyType  37402 non-null  category
 3   bedrooms      24486 non-null  float64 
 4   detailUrl     37402 non-null  object  
 5   location_lat  37402 non-null  float64 
 6   location_lng  37402 non-null  float64 
 7   property_id   37402 non-null  object  
dtypes: category(1), float64(3), int64(1), object(3)
memory usage: 2.3+ MB


Descriptive statistics of continuous variables are shown as follows.

In [6]:
properties.describe()

,Unnamed: 0,bedrooms,location_lat,location_lng
count,37402.0,24486.000000,37402.000000,37402.000000
mean,0.0,2.875194,52.868188,-2.089771
std,0.0,0.993353,1.874818,1.502522
min,0.0,0.000000,0.000000,-7.557160
25%,0.0,2.000000,51.408743,-3.066690
50%,0.0,3.000000,52.650855,-2.187560
75%,0.0,3.000000,53.775813,-0.938630
max,0.0,12.000000,60.162730,1.618240


Frequencies of each level of the variable `property type` are obtained as follows.

In [7]:
properties.propertyType.value_counts()

Semi-Detached    8783
Terraced         8535
Detached         8369
Unknown          6647
Flat             5068
Name: propertyType, dtype: int64

## A Subset of the `properties` Dataset

As random samples of images have been obtained previously, a subset of the whole `properties` dataset could hence be formulated by selecting the rows of the whole `properties` dataset corresponding to the selected samples.

In [8]:
flist_id = list(map(lambda string: string[16 : -4], flist))
properties_sub = pd.DataFrame(properties.loc[properties['property_id'].isin(flist_id)])
properties_sub = properties_sub.drop_duplicates(['location_lat', 'location_lng'])
flist_id = list(properties_sub.property_id)
flist_new = []
change_names_new = list(map(lambda x: flist_new.append(f'street_view/gsv_{x}.jpg'), flist_id))

The original data should be splitted into training and testing sets, and the testing set contains 30% of the original data.

In [9]:
flist_train, flist_test, propertyType_train, propertyType_test = train_test_split(
    flist_new, # image directories
    np.array(properties_sub.propertyType), # property types
    test_size = 0.3)

In [10]:
Img_list_train = list(map(lambda x: np.asarray(Image.open(x)), flist_train))
Img_list_test = list(map(lambda x: np.asarray(Image.open(x)), flist_test))
Img_array_train = np.asarray(Img_list_train)
Img_array_test = np.asarray(Img_list_test)

For categorical variables, one-hot encoder is introduced.

In [11]:
dummy_propertyType_train = pd.get_dummies(propertyType_train)
dummy_propertyType_test = pd.get_dummies(propertyType_test)
propertyType_test_fac = np.argmax(np.array(dummy_propertyType_test), axis = 1) 

## Multi-Class Classification Using Neural Network

### Multi-Layer Perceptron (MLP) model

In [12]:
def mlp(output_dim):

    '''
    Creates a multi-layer perceptron neural network model without hidden layers.

    Parameter:
    output_dim (int): The number of output classes.
    
    Returns:
    A compiled Keras model.
    '''

    model = Sequential()
    model.add(Rescaling(1. / 255))
    model.add(Flatten())
    model.add(Dense(10, activation = tf.nn.leaky_relu))
    model.add(Dense(output_dim, activation = tf.nn.softmax))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [13]:
MLP = mlp(dummy_propertyType_train.shape[1])
MLP.fit(Img_array_train, dummy_propertyType_train, epochs = 20, batch_size = 30)

Epoch 1/20
362/362 [==============================] - 40s 107ms/step - loss: 11.5125 - accuracy: 0.2533
Epoch 2/20
362/362 [==============================] - 38s 104ms/step - loss: 8.0760 - accuracy: 0.2685
Epoch 3/20
362/362 [==============================] - 36s 99ms/step - loss: 7.9338 - accuracy: 0.2807
Epoch 4/20
362/362 [==============================] - 35s 97ms/step - loss: 7.9875 - accuracy: 0.2871
Epoch 5/20
362/362 [==============================] - 36s 99ms/step - loss: 5.9375 - accuracy: 0.3075
Epoch 6/20
362/362 [==============================] - 37s 102ms/step - loss: 5.7216 - accuracy: 0.3111
Epoch 7/20
362/362 [==============================] - 35s 97ms/step - loss: 5.1342 - accuracy: 0.3323
Epoch 8/20
362/362 [==============================] - 38s 106ms/step - loss: 3.6873 - accuracy: 0.3621
Epoch 9/20
362/362 [==============================] - 37s 103ms/step - loss: 4.6673 - accuracy: 0.3410
Epoch 10/20
362/362 [==============================] - 36s 98ms/step - loss:

In [14]:
propertyType_test_pred_MLP = np.argmax(MLP.predict(Img_array_test), axis = 1) 
MLP.evaluate(Img_array_test, dummy_propertyType_test)

146/146 [==============================] - 7s 44ms/step - loss: 2.4114 - accuracy: 0.2723


[2.4114019870758057, 0.27227723598480225]

In [15]:
confusion_matrix(propertyType_test_fac, propertyType_test_pred_MLP)

array([[ 21, 355, 539, 121,   0],
       [  3, 450, 136,  66,   0],
       [ 12, 461, 614, 170,   1],
       [  8, 619, 410, 180,   0],
       [  2, 179, 205,  94,   0]], dtype=int64)

## Convolutional Neural Network

In [16]:
def cnn(output_dim):

    '''
    Creates a convolutional neural network model without hidden layers.

    Parameter:
    output_dim (int): The number of output classes.
    
    Returns:
    A compiled Keras model.
    '''

    model = Sequential()
    model.add(Rescaling(1. / 255))
    model.add(Conv2D(5, 2, padding = 'same', activation = tf.nn.leaky_relu))
    model.add(MaxPooling2D())  
    model.add(Dropout(.2)) 
    model.add(Flatten())
    model.add(Dense(output_dim, activation = tf.nn.softmax))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [17]:
CNN = cnn(dummy_propertyType_train.shape[1])
CNN.fit(Img_array_train, dummy_propertyType_train, epochs = 10, batch_size = 50)

Epoch 1/15
217/217 [==============================] - 164s 745ms/step - loss: 3.9791 - accuracy: 0.2945
Epoch 2/15
217/217 [==============================] - 156s 720ms/step - loss: 1.3340 - accuracy: 0.4529
Epoch 3/15
217/217 [==============================] - 156s 716ms/step - loss: 1.0172 - accuracy: 0.6255
Epoch 4/15
217/217 [==============================] - 158s 726ms/step - loss: 0.7837 - accuracy: 0.7488
Epoch 5/15
217/217 [==============================] - 157s 722ms/step - loss: 0.5977 - accuracy: 0.8300
Epoch 6/15
217/217 [==============================] - 155s 714ms/step - loss: 0.4599 - accuracy: 0.8868
Epoch 7/15
217/217 [==============================] - 157s 721ms/step - loss: 0.3326 - accuracy: 0.9339
Epoch 8/15
217/217 [==============================] - 155s 714ms/step - loss: 0.2454 - accuracy: 0.9592
Epoch 9/15
217/217 [==============================] - 157s 722ms/step - loss: 0.1904 - accuracy: 0.9739
Epoch 10/15
217/217 [==============================] - 153s 705m

In [18]:
propertyType_test_pred_CNN = np.argmax(CNN.predict(Img_array_test), axis = 1) 
CNN.evaluate(Img_array_test, dummy_propertyType_test)

146/146 [==============================] - 32s 213ms/step - loss: 2.7223 - accuracy: 0.3170


[2.7222673892974854, 0.3170469105243683]

In [19]:
confusion_matrix(propertyType_test_fac, propertyType_test_pred_CNN)

array([[590,  57, 186, 153,  50],
       [247, 157,  81, 142,  28],
       [552,  86, 322, 240,  58],
       [441, 137, 232, 341,  66],
       [192,  33, 101,  91,  63]], dtype=int64)

In [21]:
# import dill
# dill.dump_session('06151844.pkl')
# dill.load_session('06151844.pkl')